In [1]:
import warnings
warnings.filterwarnings('ignore')


import pandas as pd
import geopandas as gpd
pd.options.display.max_columns = 999

import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot, offline
offline.init_notebook_mode(connected=True)
import plotly.express as px

from mpl_toolkits.axes_grid1 import make_axes_locatable
# %pylab inline
pylab.rcParams['figure.figsize'] = (12, 12)
import seaborn as sns



# For changes in .py
# %reload_ext autoreload
# %autoreload 2

import numpy as np

import missingno as msno

stats = ['skew', 'mad', 'kurt']


import qgrid

def qg(df):
    return(qgrid.show_grid(df,show_toolbar=True, grid_options={'forceFitColumns': False}))

import itertools
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', -1)
pd.set_option('display.max_colwidth', 100)

import requests
import json
import xmltodict
import re
import urllib.request as urllib2
import io
import yaml

import math
import pysal as ps

# from pysal.esda.mapclassify import Quantiles, Equal_Interval, Fisher_Jenks



import imageio


from sklearn import preprocessing


NameError: name 'pylab' is not defined

In [262]:
def clean(df, ano, g_cols):
    df = df.drop(columns=['Unnamed: 16'])
    df.columns = pd.Series(df.columns).str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.replace(' ','_').str.lower().str.strip().tolist()    
    mask = df['orgao'].notnull()
    df = df[mask]
    
    for col in df.columns[-6:]:
        df[col] = df[col].str.replace('.','').str.replace(',','.')
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    mask = df['dotacao_atual']!=0
    df = df[mask]
    
    values = [
                '003 - RECURSOS VINCULADOS-FUNDO ESPECIAL DE DESPES',
                '002 - RECURSOS VINCULADOS ESTADUAIS',
                '005 - RECURSOS VINCULADOS FEDERAIS',
                '045 - REC.VINC.TRANSF.FEDERAL/SUPERAVIT FINANC.',
                '007 - OP.CRED.E CONTRIB.DO EXTERIOR-DOT.INIC.CR.SU',
                '042 - REC.VINC.ESTADUAIS-CRED.SUPERAVIT FINANCEIRO'
    ]

    for value in values:
        mask = df['fonte_de_recursos']!=value
        df = df[mask]
    
    
    df['sobras'] = df['dotacao_atual'] - df['empenhado'] 
#     cols = ['dotacao_atual','empenhado','sobras']
    df = df.sort_values(by='sobras', ascending=False)
    
    
    df = df.groupby(by=g_cols, as_index=False).sum().sort_values(by='sobras', ascending=False)[g_cols + ['sobras']].rename(columns={'sobras':ano,'orgao':'orgao_{}'.format(ano)})
    
#     col = []
    
    return df 

In [288]:
ano = '2015'

df = pd.read_csv('../data/orcamento_{}.csv'.format(ano), encoding="ISO-8859-9")

g_cols = ['orgao','fonte_de_recursos','funcao','acao']
df = clean(df,ano, g_cols)


In [289]:
years = [str(i) for i in range(2016,2020)]

# m_cols = g_cols
# m_cols.remove('orgao')

for year in years:
    df_year = pd.read_csv('../data/orcamento_{}.csv'.format(year), encoding="ISO-8859-9")
    g_cols = ['orgao','fonte_de_recursos','funcao','acao']
    df_year = clean(df_year,year,g_cols)
    m_cols = ['fonte_de_recursos','funcao','acao']
    df = pd.merge(df,df_year,how='outer', on=m_cols)

    df = df[np.flip(np.sort(np.array(df.columns)))]


In [292]:
df = df.sort_values(by=['2018','2017','2016','2015'], ascending=False)

In [294]:
# df.to_excel('sobras_orcamentarias_2015_a_2019.xlsx', index=False, encoding='utf-8')